In [1]:
from assets.data_generator import DR_Generator
import os
from assets.models import SMD_Unet
from assets.trainer import Trainer
import tensorflow as tf

In [2]:
masks = ['HardExudate_Masks', 'Hemohedge_Masks', 'Microaneurysms_Masks', 'SoftExudate_Masks']
mask_dir = '../data/FGADR/Seg-set'
mask_paths = [os.path.join(mask_dir, mask) for mask in masks]

generator_args = {
    'dir_path':'../data/FGADR/Seg-set/Original_Images/',
    'mask_path':mask_paths,
    'use_mask':True,
    'img_size':(512, 512),  
    'batch_size':4, # 8로 하면 바로 OOM 뜸
    'dataset':'FGADR', # FGADR or EyePacks
    'use_3channel':False,
    'CLAHE_args':None,
    'add_noise_std':None
}

tr_fgadr_gen = DR_Generator(start_end_index=(0, 1292), is_train=True, **generator_args)
val_fgadr_gen = DR_Generator(start_end_index=(1292, 1476), is_train=False, **generator_args)
ts_fgadr_gen = DR_Generator(start_end_index=(1476, 1842), is_train=False, **generator_args)

# tr_fgadr_gen = DR_Generator(start_end_index=(0, 30000), is_train=True, **generator_args)
# val_fgadr_gen = DR_Generator(start_end_index=(30000, 31000), is_train=False, **generator_args)

In [3]:
model = SMD_Unet(enc_filters=[64, 128, 256, 512, 1024], dec_filters=[512, 256, 64, 32], input_channel=1)

In [4]:
model.load_weights('../models/withoutCLAHE_pretrained_1channel')

In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01) 

trainer_args = {
    'model':model,
    'epochs':100,
    'optimizer':optimizer,
    'for_recons':False,
    'alpha':0.05,
    'beta':None,
    'first_epoch':1,
    'file_name':'history/withoutCLAHE_pretrained_1channel_lr001_alp005.txt',
    'save_model_path':'../models/withoutCLAHE_pretrained_1channel_lr001_alp005',
    'add_noise':False
}
trainer = Trainer(**trainer_args)

history = trainer.train(train_dataset=tr_fgadr_gen,
                        val_dataset=val_fgadr_gen,
                       )


Epoch 1/100
184/184 [==============================] - 42s 226ms/step - val_loss: 0.9144 - mask_loss: 0.9608 - loss_recons: 0.0334

Epoch 2/100
 188/1292 [===>..........................] - ETA: 6:13 - train_loss: 0.9278 - mask_loss: 0.9743 - loss_recons: 0.0446

KeyboardInterrupt: 